# Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
import time
np.random.seed(606)

In [2]:
df = pd.read_csv("dataset-logistic-regression.csv")

X = np.array(df.iloc[:, 1:])
y = np.array(df["y"])

In [3]:
def sigmoid(z):
    #denom = 1 + np.exp(-z)
    return 1 / (1 + np.exp(-z))

#error functions
def error(X, y, beta):
    pred = np.clip(sigmoid(X @ beta), 1e-10, 1 - 1e-10)
    return -np.mean(y * np.log(pred) + (1 - y) * np.log(pred))

def grad_error(X, y, beta):
    pred = sigmoid(X @ beta)
    return (1 / X.shape[0]) * X.T @ (pred - y)
    

#SGD Implementation
def error_SGD(X, y, w, n):
    left = np.clip(y[n] * np.log(sigmoid(w.T @ X[n])), 1e-10, 1 - 1e-10) #logreg for observation n when actual = 1
    right = np.clip((1 - y[n]) * np.log(1 - sigmoid(w.T @ X[n])), 1e-10, 1 - 1e-10) #logreg for observation n when actual = 0
    return -sum(left + right)

def grad_error_SGD(X, y, w, n):
    return (sigmoid(w.T @ X[n]) - y[n]) * X[n] #gradient of logit for one observation

In [147]:
#Logistic Regresssion check
lgs = LogisticRegression(fit_intercept=False, solver='lbfgs')
lgs.fit(X, y)
beta_lgs = lgs.coef_.flatten()  # shape match

# Gradient Descent with Backtracking Line Search

In [154]:
def backtracking(X, y, eta, tol, epsilon, tau, max_iter=1000):
    n, p = X.shape
    beta = np.random.randn(p) * 0.01
    i = 0
    initial_eta = eta  # Save original eta

    for _ in range(max_iter):
        i += 1
        grad = grad_error(X, y, beta)
        eta = initial_eta  # Reset eta at the start of each iteration
        beta_new = beta - eta * grad

        while error(X, y, beta_new) > error(X, y, beta) - epsilon * eta * np.sum(grad**2): #check
            eta *= tau
            if eta < 10e-3: #accounts for if eta shrinks too much
                break
            beta_new = beta - eta * grad

        if np.linalg.norm(beta_new - beta) < tol:
            break
        
        beta = beta_new

    return beta_new, i

beta_bls, iter_bls = backtracking(X, y, 1, 1e-6, 0.1, 0.8, 1000) 

The step size ($\eta$) used should be a large step size. $\epsilon$ and $\tau$ are chosen from $(0, 1)$ but to prevent vanishing gradient by the backtracking algorithm, I set $\epsilon = 0.1$ and $\tau = 0.8$.

In [178]:
e1 = np.linalg.norm(beta_lgs - beta_bls)
print(e1)

0.38511601688783237


# Nesterov Gradient Descent with Backtracking Line Search

In [159]:
def nesterov(X, y, eta, tol, epsilon, tau, xi, max_iter=1000):
    n, p = X.shape
    beta_first = np.random.randn(p) * 0.01
    i = 0
    initial_eta = eta  # Save original eta
    beta_second = beta_first - eta * grad_error(X, y, beta_first)

    for _ in range(1, max_iter):
        i += 1
        lookahead = beta_second + xi * (beta_second - beta_first)
        grad = grad_error(X, y, lookahead)
        eta = initial_eta  # Reset eta at the start of each iteration
        beta_new = lookahead - eta * grad

        while error(X, y, beta_new) > error(X, y, lookahead) - epsilon * eta * np.sum(grad**2): #check
            eta *= tau
            beta_new = lookahead - eta * grad

        if np.linalg.norm(beta_new - beta) < tol:
            break
        
        beta_first = beta_second
        beta_second = beta_new

    return beta_new, i

beta_nesterov, iter_nesterov = nesterov(X, y, 1, 1e-6, 1e-2, 0.5, 0.5, 1000) 

The parameters in the armijo's condition are the same as the algorithm coded above. The momentum parameter $\xi \in (0, 1)$ will be chosen as $0.5$ to keep it in between.

In [179]:
e2 = np.linalg.norm(beta_nesterov - beta_lgs)
print(e2)

0.33241061892955603


# AMSGrad-ADAM Momentum

In [161]:
def amsgrad_adam(X, y, eta, beta1=0.9, beta2=0.999, epsilon=1e-8, max_iter=1000, tol = 1e-6):
    x = np.zeros(X.shape[1])
    m = np.zeros_like(x)  #moment m
    z = np.zeros_like(x)  #moment z
    z_hat = z #z_hat to compare for max
    i = 0
    
    for k in range(1, max_iter + 1):
        i += 1 #iteration
        g = grad_error(X, y, x)  #Compute gradient Gk

        #Update m and z
        m = beta1 * m + (1 - beta1) * g
        z = beta2 * z + (1 - beta2) * (g**2)

        #m_hat
        m_hat = m / (1 - beta1 ** k)

        #find max of z_hat and z
        z_hat = np.maximum(z_hat, z)

        # Normalize z
        z_tilde = 1 / (np.sqrt(z_hat) + epsilon)

        # Update parameters
        x_new = x - eta * (z_tilde * m_hat)
        
        if np.linalg.norm(x_new - x) < tol:
            break
        x = x_new


    return x, iter

beta_ams, iter_ams = amsgrad_adam(X, y, eta = 0.01, beta1 = 0.9, beta2 = 0.99, epsilon = 1e-7, tol = 1e-6)

I set the step size $\eta = 0.01$ as this algorithm needs a small initial step size. Since $\beta_2 \in (0, 1)$ and $\beta_1 \in (0, \beta_2)$, I initially chose $\beta_2$ to be closer to 1 to emphasize the momentums and similarly for $\beta_1$ but less than $\beta_2$ to keep satisfy the restrictions. $\epsilon$ is set to a small value to prevent vanishing gradient. 

In [180]:
e3 = np.linalg.norm(beta_lgs - beta_ams)
print(e3)

0.0013357059687899048


# SGD with Decreasing Step Size

In [151]:
def minibatch(X, y, batchsize, iter = 1000, step = 0.01, scale = 0.0001, tol = 1e-6):
    #initialize beta0
    n, p = X.shape
    beta = np.random.randn(p) * 0.01
    i = 0

    for _ in range(iter):
        #set the diminishing step size???
        i += 1
        step_new = step /  (1 + scale * i)
        #set the error term from the minibatch
        batch = np.random.randint(0, n - 1, batchsize)
        target = 0
        for j in batch:
            target += grad_error_SGD(X, y, beta, j) #update error on summation of batch
        target = target / batchsize

        beta_new = beta - (step_new * target)
        if np.linalg.norm(beta_new - beta) < tol:
            break
        beta = beta_new

    

    return beta, i


Batch size is determined by the given values the homework assigns. The step size is set to a small value to ensure convergence as the gradient can vary based on which observations are chosen for the SGD iteration. The scale for the diminishing step size is also set to a small value to ensure the step size isn't too small, which could lead to a vanishing gradient if it is too small.

In [172]:
# Batch size = 100
beta_mini_100, iter_mini_100 = minibatch(X, y, batchsize = 100, step = 0.1, iter = 1000)

# Batch Size = 200
beta_mini_200, iter_mini_200 = minibatch(X, y, batchsize = 200, step = 0.1, iter = 1000)

# Batch Size = 500
beta_mini_500, iter_mini_500 = minibatch(X, y, batchsize = 500, step = 0.1, iter = 1000)

In [181]:
# Batch size = 100
e4 = np.linalg.norm(beta_mini_100 - beta_lgs)
print(e4)

# Batch size = 200
e5 = np.linalg.norm(beta_mini_200 - beta_lgs)
print(e5)

# Batch Size = 500
e6 = np.linalg.norm(beta_mini_500 - beta_lgs)
print(e6)

0.19096485542772365
0.16441518898455296
0.09662314721147974


# SGD AMSGrad-ADAM Momentum

In [129]:
def adamw(X, y, eta, batchsize, beta1=0.9, beta2=0.999, epsilon=1e-8, lmb = 0.1, max_iter=1000, tol = 1e-6):
    n, p = X.shape
    x = np.zeros(p)

    m = np.zeros_like(x)  #moment m
    z = np.zeros_like(x)  #moment z
    z_hat = z #z_hat to compare for max
    i = 0
    
    for k in range(1, max_iter + 1):
        i += 1 #iteration
        #set the error term from the minibatch
        batch = np.random.randint(0, n - 1, batchsize)
        g = 0
        for j in batch:
            g += grad_error_SGD(X, y, x, j) #update error on summation of batch
        g = g/batchsize

        #Update m and z
        m = beta1 * m + (1 - beta1) * g
        z = beta2 * z + (1 - beta2) * (g * g)

        #m_hat
        m_hat = m / (1 - beta1 ** k)

        #find max of z_hat and z
        z_hat = np.maximum(z_hat, z)

        # Normalize z
        z_tilde = 1 / (np.sqrt(z_hat) + epsilon)

        # Update parameters
        x = (1 - lmb*eta)*x - eta * (z_tilde * m_hat)
        
        #if np.linalg.norm(x


    return x, iter


All the parameters are similar to the parameters from SGD and AMSGrad-Adam. The only new parameter is $\lambda$, the scale size for the weight decay. From the slides, the $\lambda$ value should be set up to $10^{-2}$ as it is a small model.

In [174]:
# Batch Size = 100
beta_adamw_100, iter_ams_100 = adamw(X, y, eta = 0.1, batchsize = 100)

# Batch Size = 200
beta_adamw_200, iter_ams_200 = adamw(X, y, eta = 0.1, batchsize = 200)

# Batch Size = 500
beta_adamw_500, iter_ams_500 = adamw(X, y, eta = 0.1, batchsize = 500)

C:\Users\julia\AppData\Local\Temp\ipykernel_13140\3475053029.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [182]:
# Batch size = 100
e7 = np.linalg.norm(beta_adamw_100 - beta_lgs)
print(e7)

# Batch size = 200
e8 = np.linalg.norm(beta_adamw_200 - beta_lgs)
print(e8)

# Batch Size = 500
e9 = np.linalg.norm(beta_adamw_500 - beta_lgs)
print(e9)

0.7793445616362327
0.7153749717658412
0.44986693439729647


# Table of Values

In [189]:
errors = [e1, e2, e3, e4, e5, e6, e7, e8, e9]
iterations = [iter_bls, iter_nesterov, iter_ams, iter_mini_100, iter_mini_200, iter_mini_500, iter_ams_100, iter_ams_200, iter_ams_500]

# Create Index
index_names = [
    "Backtracking Line Search",
    "Nesterov BLS",
    "AMSGrad-ADAM",
    "SGD 100",
    "SGD 200",
    "SGD 500",
    "AMSGrad-ADAMw 100",
    "AMSGrad-ADAMw 200",
    "AMSGrad-ADAMw 500"
]

# Create table
table = pd.DataFrame({
    'Estimation Error': errors,
    'Iterations': iterations
}, index = index_names)

print(table)


                          Estimation Error  Iterations
Backtracking Line Search          0.385116        1000
Nesterov BLS                      0.332411         999
AMSGrad-ADAM                      0.001336        1000
SGD 100                           0.190965        1000
SGD 200                           0.164415        1000
SGD 500                           0.096623        1000
AMSGrad-ADAMw 100                 0.779345        1000
AMSGrad-ADAMw 200                 0.715375        1000
AMSGrad-ADAMw 500                 0.449867        1000


- AMSGrad-ADAM seems to perform the best in my algorithm while AMSGrad-ADAMw 100 seems to perform the worst
- The batch size may not be enough for the AMSGrad-ADAMw algorithm as the algorithm may be highly sensitive to the gradients the batch size selects as these gradients determine the weights
- Maybe the weight decay is too small but wouldn't want to adjust as it may end up overfitting the data
- The Nesterov BLS algorithm took the longest to run, while the SGD 100 took the shortest
- I had to cap the eta for my BLS algorithm as it kept shrinking too fast, leaving my gradient to vanish and my algorithm to converge too fast
- Many of my hyperparameters may be fine tuned to get a better estimation error but with runtime conflicts and the information given during lecture, these were the values I was able to get
- I capped all iterations to 1000 and all the algorithms seem to require the max iterations
- Overall, I understand why ADAM operation is widely used for neural networks, it had a decently fast convergence rate compared to the other algorithms and performed the best